# Prepare Code to be Run on MTP Nodes

## Input Required Information

In [ ]:
SLICE_NAME = "Clos"
CONFIG_FILE = "node_config.json"
CODE_DIR = "/home/fabric/work/custom/CMTP/SRC"

## Access the Slice

The orchestrator class is initalized, which also means the slice and its nodes are now accessable as well.

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")

## Upload the Source Code

MAKE SURE THAT YOU UPDATE mtp_dcn.conf BECAUSE IT GETS OVERWRITTEN AND THEN SEG FAULTS

In [ ]:
manager.uploadDirectoryParallel(CODE_DIR, prefixList="L,S")

## Compile the Code

In [ ]:
compileCmd = "cd ~/SRC ; gcc main.c mtp_build.c mtp_send.c mtp_struct.c mtp_utils.c -o MTPstart"
manager.executeCommandsParallel(compileCmd, prefixList="L,S")

## Configure Nodes Based on Their folded-Clos Tier and Role

This is where the difference between a leaf, spine, and a top-tier spine becomes important, as each has different configuration needs that cannot be swapped or ignored.

In [ ]:
import json
import time

def get_node_tiers(data):
    node_tier_list = []

    queue = [data['topology']]
    
    while len(queue) != 0:
        temp_list = []
        while len(queue) != 0:
            temp_list.append(queue.pop(0))

        tot = []
        for pod in temp_list:
            for spine in pod['topSpines']:
                tot.append(spine)
        
        node_tier_list.append(tot)
            
        for pod in temp_list:
            if "pods" in pod:
                for pod_t in pod["pods"]:
                    queue.append(pod_t)
    
    node_tier_list.append(data['topology']['leaves'])

    return node_tier_list

# Open the configuration file and read it
f = open("node_config.json")
data = json.load(f)
f.close()

# Get the information
node_tier_list = get_node_tiers(data)

# Get start time info
start_hello_time = round(time.time()*1000) + 3 * 60 * 1000

for i in range(len(node_tier_list)):
    for currentRemoteNode in node_tier_list[i]:
        print(f"Configuring {currentRemoteNode}")
        
        # If it is a leaf
        if i == len(node_tier_list) - 1:
            isTor = "true"
            isTopSpine = "false"
            tier = 1
            torEthPortName = data['topology']['leavesNetworkPortDict'][currentRemoteNode]
        
        # If it is a spine
        else:
            isTor = "false"
            tier = len(node_tier_list) - i
            torEthPortName = "0"
            
            # If is a top-tier spine
            if i == 0:
                isTopSpine = "true"
            
            # If it is not a top-tier spine
            else:
                isTopSpine = "false"
                
        configCmd = f"cd ~/SRC; sudo bash config_generator.sh {isTor} {isTopSpine} {tier} {start_hello_time} {torEthPortName}"
        
        # Execute the script remotely
        manager.executeCommandsParallel(configCmd, prefixList=currentRemoteNode)